# Assignment: 1 - Camera Calibration 

### Importing necessary libraries

In [234]:
import numpy as np
import pandas as pd
from scipy import linalg

### Normalization

In [235]:
def Normalization(nd, x):
    # CONVERTING TO NUMPY ARRAY
    x = np.asarray(x)
    # print(x)

    # CALCULATING centroid and mean distance from centroid
    m = np.mean(x, 0)
    dist = np.mean(np.sqrt(np.sum(np.square(x - m))))

    # NORMALIZATION MATRIX FOR WORLD POINTS(3D) AND IMAGE-PIXEL POINTS(2D)
    if nd == 2:
        s2D = np.sqrt(2) / dist
        Tr = np.diag([s2D, s2D, 1])
        Tr[0:2, 2] = -m * s2D

    else:
        s3D = np.sqrt(3) / dist
        Tr = np.diag([s3D, s3D, s3D, 1])
        Tr[0:3, 3] = -m * s3D

    x = np.dot(Tr, np.concatenate((x.T, np.ones((1, x.shape[0])))))
    x = x[0:nd, :].T

    print("MEAN DISTANCE FROM CENTER : {}".format(np.mean(np.sqrt(np.sum(np.square(x))))))

    return Tr, x

### DLT Camera Calibration

In [236]:
def DLTcalib(nd, xyz, img_pt_):

    if (nd != 3):
        raise ValueError('%dD DLT unsupported.' %(nd))
    
    # Converting all variables to numpy array
    xyz = np.asarray(xyz)
    img_pt_ = np.asarray(img_pt_)

    n = xyz.shape[0]

    print("\n\nNUMBER OF POINTS: {}\n\n".format(n))
    # Validating the parameters:
    if img_pt_.shape[0] != n:
        raise ValueError('Object (%d points) and image (%d points) have different number of points.' %(n, img_pt_.shape[0]))

    if (xyz.shape[1] != 3):
        raise ValueError('Incorrect number of coordinates (%d) for %dD DLT (it should be %d).' %(xyz.shape[1],nd,nd))

    if (n < 6):
        raise ValueError('%dD DLT requires at least %d calibration points. Only %d points were entered.' %(nd, 2*nd, n))
        
    # Normalize the data to improve the DLT quality (DLT is dependent of the system of coordinates).
    # This is relevant when there is a considerable perspective distortion.
    # Normalization: mean position at origin and mean distance equals to 1 at each direction.
    Txyz, xyzn = Norm(nd, xyz)
    Timg_pt_, img_pt_n = Norm(2, img_pt_)

    A = []

    for i in range(n):
        x, y, z = xyzn[i, 0], xyzn[i, 1], xyzn[i, 2]
        u, v = img_pt_n[i, 0], img_pt_n[i, 1]
        A.append( [x, y, z, 1, 0, 0, 0, 0, -u * x, -u * y, -u * z, -u] )
        A.append( [0, 0, 0, 0, x, y, z, 1, -v * x, -v * y, -v * z, -v] )

    # Convert A to array
    A = np.asarray(A) 

    # Find the 11 parameters:
    U, S, V = np.linalg.svd(A)

    # The parameters are in the last line of Vh and normalize them
    L = V[-1, :] / V[-1, -1]

    # Camera projection matrix
    H = L.reshape(3, nd + 1)


    # Denormalization
    H = np.dot( np.dot( np.linalg.pinv(Timg_pt_), H ), Txyz )
    H = H / H[-1, -1]
    L = H.flatten()

    # Mean error of the DLT (mean residual of the DLT transformation in units of camera coordinates):
    img_pt_2 = np.dot( H, np.concatenate( (xyz.T, np.ones((1, xyz.shape[0]))) ) ) 
    img_pt_2 = img_pt_2 / img_pt_2[2, :] 

    # Mean distance:
    projected_pts = img_pt_2.T
    actual_pixels = img_pt_

    # Printing teh actual and the estimated points
    print('\n\nACTUAL PTS   |             ESTIMATED PTS')
    for i in range(len(img_pt_2.T)):
        print("{}   <---->  {}".format(actual_pixels[i], projected_pts[i]))

    err = np.sqrt( np.mean(np.sum( (img_pt_2[0:2, :].T - img_pt_)**2, 1)) ) 

    return L, err

In [237]:
def get_intrest_points():

    # WORLD CORDINATES
    data = pd.read_excel("dataset.xlsx")
    x = np.array(data['X'])
    y = np.array(data['Y'])
    z = np.array(data['Z'])

    # IMAGE PIXEL COORDINATES
    u = np.array(data['x'])
    v= np.array(data['y'])

    # LIST OF WORLD PTS AND CORRESPONDING PIXEL PTS
    world_pts = []
    pixel_pts = []

    for i in range(len(x)):
        world_pts.append([x[i], y[i], z[i]])
        pixel_pts.append([u[i], v[i]])

    print("DATA FRAME")
    print(data)

    # from sklearn.utils import shuffle
    # world_pts, pixel_pts = shuffle(world_pts, pixel_pts, random_state = 0)

    print("\n\nWORLD points")
    print(world_pts)
    return world_pts, pixel_pts
   

### RQ Decomposition 

In [238]:
def RQ_decomposition(P):
    M = P[0:3,0:3]
    print("\nM:\n",M)
    K, R = linalg.rq(M)
    T = np.diag(np.sign(np.diag(K)))
    print(T)

    K = np.dot(K, T)
    R = np.dot(T, R)
    C = np.dot(linalg.inv(-M), P[:, 3])
    return K, R, C


In [239]:
    def camera_param(P):
        print("\nP:\n",P)

        K, R, C = RQ_decomposition(P)

        print("\n\n K MATRIX: ")
        print(K)
        print("\n\n R MATRIX: ")
        print(R)
        print("\n\n Camera center: ")
        print(C)

        print("\n\n Normalized camera matrix: ")
        print(K/K[2][2])


In [240]:
#camera_param(P)

In [241]:

xyz, img_pt_ = get_intrest_points()
nd = 3
P, err = DLTcalib(nd, xyz, img_pt_)
P = P.reshape(3,4)

DATA FRAME
       X     Y   Z    x     y  Unnamed: 5
0   20.0   0.0   0  708   991           1
1    0.0  20.0   0  213   998           2
2    0.0   0.0  20  455   597           3
3   20.0  20.0   0  464  1192           4
4   20.0   0.0  20  727   658           5
5    0.0  20.0  20  182   665           6
6    0.0  15.0  10  279   810           7
7    0.0   7.5  15  373   699           8
8   12.5  17.5   0  396  1082           9
9    5.0   0.0   5  516   828          10
10  10.0  10.0   0  463   998          11
11  10.0   0.0  10  576   779          12
12   0.0  10.0  10  343   784          13
13   0.0   0.0   0  463   874          14
14  15.0  10.0   0  529  1038          15
15  15.0   0.0  10  642   805          16
16  25.0   0.0  25  829   576          17
17   0.0  25.0  25   76   586          18


WORLD points
[[20.0, 0.0, 0], [0.0, 20.0, 0], [0.0, 0.0, 20], [20.0, 20.0, 0], [20.0, 0.0, 20], [0.0, 20.0, 20], [0.0, 15.0, 10], [0.0, 7.5, 15], [12.5, 17.5, 0], [5.0, 0.0, 5], [10.0, 10.0

### Projection matrix (P)

In [242]:
print('Matrix')
print(P)

Matrix
[[ 5.69272651e+00 -1.44197424e+01 -2.04513781e+00  4.62512816e+02]
 [-3.38023576e+00 -3.25018551e+00 -1.60975813e+01  8.72708150e+02]
 [-9.39329037e-03 -9.60097812e-03 -3.83988885e-03  1.00000000e+00]]


### RMSE

In [243]:
print('\nError')
print(err)


Error
2.0692087872744303


### Camera Parameters

In [244]:
camera_param(P.reshape(3,4))


P:
 [[ 5.69272651e+00 -1.44197424e+01 -2.04513781e+00  4.62512816e+02]
 [-3.38023576e+00 -3.25018551e+00 -1.60975813e+01  8.72708150e+02]
 [-9.39329037e-03 -9.60097812e-03 -3.83988885e-03  1.00000000e+00]]

M:
 [[ 5.69272651e+00 -1.44197424e+01 -2.04513781e+00]
 [-3.38023576e+00 -3.25018551e+00 -1.60975813e+01]
 [-9.39329037e-03 -9.60097812e-03 -3.83988885e-03]]
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]


 K MATRIX: 
[[1.41549182e+01 8.46664248e-02 6.64453293e+00]
 [0.00000000e+00 1.41899036e+01 8.93131910e+00]
 [0.00000000e+00 0.00000000e+00 1.39698758e-02]]


 R MATRIX: 
[[ 0.7166994  -0.69731471 -0.00970399]
 [ 0.18500116  0.20352353 -0.96143265]
 [-0.67239612 -0.68726295 -0.27486922]]


 Camera center: 
[45.82396051 45.13378285 35.47857168]


 Normalized camera matrix: 
[[1.01324581e+03 6.06064261e+00 4.75632927e+02]
 [0.00000000e+00 1.01575016e+03 6.39327021e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [245]:
import cv2 as cv

In [246]:
img = cv.imread('THREE_PLANE_DATA/image (12).jpg')
cv.imshow('orginal image', img)

In [247]:
Norm(3, xyz)

(array([[ 0.02661088,  0.        ,  0.        , -0.22545325],
        [ 0.        ,  0.02661088,  0.        , -0.2291492 ],
        [ 0.        ,  0.        ,  0.02661088, -0.25132493],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.30676426, -0.2291492 , -0.25132493],
        [-0.22545325,  0.3030683 , -0.25132493],
        [-0.22545325, -0.2291492 ,  0.28089257],
        [ 0.30676426,  0.3030683 , -0.25132493],
        [ 0.30676426, -0.2291492 ,  0.28089257],
        [-0.22545325,  0.3030683 ,  0.28089257],
        [-0.22545325,  0.17001392,  0.01478382],
        [-0.22545325, -0.02956764,  0.1478382 ],
        [ 0.10718269,  0.23654111, -0.25132493],
        [-0.09239887, -0.2291492 , -0.11827056],
        [ 0.0406555 ,  0.03695955, -0.25132493],
        [ 0.0406555 , -0.2291492 ,  0.01478382],
        [-0.22545325,  0.03695955,  0.01478382],
        [-0.22545325, -0.2291492 , -0.25132493],
        [ 0.17370988,  0.03695955, -0.25132493],
        [ 0.173